In [2]:
import os
print(os.getcwd())

c:\Users\v0166\Github\Introduction_to_AI_Homework\src\hw4


In [8]:
import csv

# 讀取 CSV 檔案的函式
def load_csv(filename):
    data = []
    with open(filename, mode='r', encoding='utf-8-sig') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data.append(row)
    return header, data

# 測試讀取 train.csv, val.csv 和 test.csv
train_header, train_data = load_csv('../../res/hw4/train.csv')
train_gt_header, train_gt_data = load_csv('../../res/hw4/train_gt.csv')
val_header, val_data = load_csv('../../res/hw4/val.csv')
val_gt_header, val_gt_data = load_csv('../../res/hw4/val_gt.csv')
test_header, test_data = load_csv('../../res/hw4/test.csv')

# 簡單檢視讀取結果
print("Train Data Sample:", train_data[:2])  # 取前兩筆資料查看
print("Train Ground Truth Sample:", train_gt_data[:2])
print("Validation Data Sample:", val_data[:2])
print("Validation Ground Truth Sample:", val_gt_data[:2])
print("Test Data Sample:", test_data[:2])

Train Data Sample: [['Male', 'No', 'Yes', 'Yes', '27', 'Yes', 'Yes', 'Fiber optic', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Month-to-month', 'Yes', 'Credit card (automatic)', '85.2', '2151.6'], ['Female', 'No', 'No', 'No', '5', 'Yes', 'Yes', 'Fiber optic', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Month-to-month', 'Yes', 'Electronic check', '90.8', '455.5']]
Train Ground Truth Sample: [['No'], ['Yes']]
Validation Data Sample: [['Female', 'Yes', 'No', 'No', '26', 'Yes', 'Yes', 'Fiber optic', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Month-to-month', 'Yes', 'Electronic check', '95.0', '2497.2'], ['Male', 'No', 'No', 'No', '42', 'Yes', 'No', 'No', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'One year', 'No', 'Bank transfer (automatic)', '19.8', '849.9']]
Validation Ground Truth Sample: [['Yes'], ['No']]
Test Data Sample: [['Male', 'Yes', 'No', 'No', '10', 'Yes', 'No', 'Fiber optic', 'No', 'No', 'No', '

In [9]:
import numpy as np

# 資料清理與轉換函式
def preprocess_data(data):
    processed_data = []
    for row in data:
        # 轉換性別為數值型 (Male: 0, Female: 1)
        gender = 0 if row[0] == "Male" else 1
        
        # 轉換是否為老年人 (SeniorCitizen: 0, 1)
        senior_citizen = 1 if row[1] == "Yes" else 0
        
        # 轉換是否有配偶 (Partner: Yes -> 1, No -> 0)
        partner = 1 if row[2] == "Yes" else 0
        
        # 轉換是否有供養對象 (Dependents: Yes -> 1, No -> 0)
        dependents = 1 if row[3] == "Yes" else 0
        
        # 服務年限 (tenure): 保持原數值
        tenure = int(row[4])
        
        # 轉換電話服務 (PhoneService: Yes -> 1, No -> 0)
        phone_service = 1 if row[5] == "Yes" else 0
        
        # 轉換多條電話線 (MultipleLines: Yes -> 1, No -> 0, No phone service -> 0)
        multiple_lines = 1 if row[6] == "Yes" else 0
        
        # 進行 One-hot Encoding: InternetService (Fiber optic, DSL, No)
        internet_service = [0, 0]  # Fiber optic, DSL
        if row[7] == "Fiber optic":
            internet_service[0] = 1
        elif row[7] == "DSL":
            internet_service[1] = 1
        
        # 轉換 OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies
        # 將 "No internet service" 視為 0
        online_security = 1 if row[8] == "Yes" else 0
        online_backup = 1 if row[9] == "Yes" else 0
        device_protection = 1 if row[10] == "Yes" else 0
        tech_support = 1 if row[11] == "Yes" else 0
        streaming_tv = 1 if row[12] == "Yes" else 0
        streaming_movies = 1 if row[13] == "Yes" else 0
        
        # 進行 One-hot Encoding: Contract (One year, Two year, Month-to-month)
        contract = [0, 0]  # One year, Two year
        if row[14] == "One year":
            contract[0] = 1
        elif row[14] == "Two year":
            contract[1] = 1
        
        # 轉換 PaperlessBilling (Yes -> 1, No -> 0)
        paperless_billing = 1 if row[15] == "Yes" else 0
        
        # 進行 One-hot Encoding: PaymentMethod
        payment_method = [0, 0, 0]  # Electronic check, Bank transfer, Credit card
        if row[16] == "Electronic check":
            payment_method[0] = 1
        elif row[16] == "Bank transfer (automatic)":
            payment_method[1] = 1
        elif row[16] == "Credit card (automatic)":
            payment_method[2] = 1
        
        # 處理數值特徵: MonthlyCharges, TotalCharges
        monthly_charges = float(row[17])
        total_charges = float(row[18]) if row[18] != "" else 0.0  # 處理空值
        
        # 組合所有特徵
        processed_data.append([
            gender, senior_citizen, partner, dependents, tenure,
            phone_service, multiple_lines, *internet_service,
            online_security, online_backup, device_protection,
            tech_support, streaming_tv, streaming_movies, 
            *contract, paperless_billing, *payment_method,
            monthly_charges, total_charges
        ])
    
    return np.array(processed_data)

# 進行前處理
processed_train_data = preprocess_data(train_data)
processed_val_data = preprocess_data(val_data)
processed_test_data = preprocess_data(test_data)

# 顯示前處理後的資料前兩筆
print("Processed Train Data Sample:", processed_train_data[:2])


Processed Train Data Sample: [[0.0000e+00 0.0000e+00 1.0000e+00 1.0000e+00 2.7000e+01 1.0000e+00
  1.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00
  0.0000e+00 0.0000e+00 1.0000e+00 8.5200e+01 2.1516e+03]
 [1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 5.0000e+00 1.0000e+00
  1.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00
  0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 9.0800e+01 4.5550e+02]]


In [10]:
import numpy as np
from collections import Counter

# 距離計算函式 - 使用歐幾里得距離
def calculate_distance(sample1, sample2):
    return np.sqrt(np.sum((sample1 - sample2) ** 2))

# KNN 演算法函式
def knn_predict(train_data, train_labels, test_data, k=5):
    predictions = []
    for test_point in test_data:
        distances = []
        
        # 計算每一個訓練樣本與測試樣本之間的距離
        for i, train_point in enumerate(train_data):
            distance = calculate_distance(test_point, train_point)
            distances.append((distance, train_labels[i]))  # (距離, 標籤)
        
        # 根據距離排序，取前 K 個最近的樣本
        distances.sort(key=lambda x: x[0])
        k_nearest_neighbors = distances[:k]
        
        # 取得 K 個鄰居的標籤進行投票
        k_nearest_labels = [label for _, label in k_nearest_neighbors]
        most_common_label = Counter(k_nearest_labels).most_common(1)[0][0]
        
        # 儲存預測結果
        predictions.append(most_common_label)
    
    return predictions

# 評估模型準確率
def evaluate_accuracy(predictions, ground_truth):
    correct = sum([1 for pred, actual in zip(predictions, ground_truth) if pred == actual[0]])  # 修正這裡
    accuracy = correct / len(ground_truth)
    return accuracy

# 使用 KNN 進行驗證集預測
k = 3  # 可以嘗試調整 K 值
val_predictions = knn_predict(processed_train_data, [label[0] for label in train_gt_data], processed_val_data, k)

# 計算驗證集的準確率
accuracy = evaluate_accuracy(val_predictions, val_gt_data)
print(f"Validation Accuracy: {accuracy:.4f}")

# 預測結果檢視（前 10 筆）
print("Validation Predictions Sample:", val_predictions[:10])


Validation Accuracy: 0.6845
Validation Predictions Sample: ['Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']
